In [38]:
# using Pkg
# Pkg.add("JSON") 

In [39]:
using DataStructures

In [40]:
lens = Dict{Tuple{Int64, Int64}, Real}()

# for i in 1:4
#     for j in 1:4
#         ind = (i, j)
#         if i == j
#             lens[ind] = 0
#         else
#             lens[ind] = 1
#         end
#     end
# end

eps = 1e-9

1.0e-9

In [41]:
function cross_prod(v::AbstractVector{<:Real}, w::AbstractVector{<:Real})
    return v[1] * w[2] - v[2] * w[1]
end

function quadratic_equation_solve(a::Real, b::Real, c::Real)
    D = b * b - 4 * a * c
    return [(-b - sqrt(D)) / (2 * a), (-b + sqrt(D)) / (2 * a)]
end

function dist(v::AbstractVector{<:Real}, w::AbstractVector{<:Real})
    return sqrt((v[1] - w[1]) * (v[1] - w[1]) + (v[2] - w[2]) * (v[2] - w[2]))
end

function dist3(v::AbstractVector{<:Real}, w::AbstractVector{<:Real})
    return sqrt((v[1] - w[1]) * (v[1] - w[1]) + (v[2] - w[2]) * (v[2] - w[2]) + (v[3] - w[3]) * (v[3] - w[3]))
end

dist3 (generic function with 1 method)

In [42]:
function intersection(w1::Tuple{AbstractVector{<:Real}, <:Real}, w2::Tuple{AbstractVector{<:Real}, <:Real})
    x1 = w1[1][1]
    x2 = w2[1][1]
    y1 = w1[1][2]
    y2 = w2[1][2]
    r1 = w1[2]
    r2 = w2[2]

    a = 2 * (x2 - x1)
    b = 2 * (y2 - y1)
    c = (x1 * x1 - x2 * x2 + y1 * y1 - y2 * y2 - r1 * r1 + r2 * r2)
    println(b)
    if abs(b) < eps
        println("q")
        x = -c / a
        ys = quadratic_equation_solve(1, -2 * y1, y1 * y1 - r1 * r1 + (x - x1) * (x - x1))
        xs = [x, x]
    else
        println("p")
        xs = quadratic_equation_solve(((a * a) / (b * b)) + 1, -2 * x1 + 2 * (a / b) * (y1 + c / b), x1 * x1 - r1 * r1 + (y1 + c / b) * (y1 + c / b))
        ys = [(-c - a * xs[1]) / b, (-c - a * xs[2]) / b]
    end
    # println((xs[1] - x1) * (xs[1] - x1) + (ys[1] - y1) * (ys[1] - y1) - r1 * r1, ' ', (xs[1] - x2) * (xs[1] - x2) + (ys[1] - y2) * (ys[1] - y2) - r2 * r2)
    return [[xs[1], ys[1]], [xs[2], ys[2]]]
end

intersection (generic function with 1 method)

In [43]:
function make_next(A::Tuple{AbstractVector{<:Real}, Int64}, B::Tuple{AbstractVector{<:Real}, Int64}, C::Tuple{AbstractVector{<:Real}, Int64}, edge::Tuple{Int64, Int64})
    left = edge[1]
    right = edge[2]
    vertexes = [A, B, C]
    ver_nums = [A[2], B[2], C[2]]
    V1 = vertexes[(findfirst(x -> x == left, ver_nums))]
    V2 = vertexes[(findfirst(x -> x == right, ver_nums))]
    V3 = vertexes[(findfirst(x -> !(x == left || x == right), ver_nums))]
    num_D = 10 - A[2] - B[2] - C[2]
    V1D = lens[(V1[2], num_D)]
    V2D = lens[(V2[2], num_D)]
    Ds = intersection((V1[1], V1D), (V2[1], V2D))
    D1 = Ds[1]
    D2 = Ds[2]
    V1V2 = [V2[1][1] - V1[1][1], V2[1][2] - V1[1][2]]
    V1D1 = [D1[1] - V1[1][1], D1[2] - V1[1][2]]
    V1V3 = [V3[1][1] - V1[1][1], V3[1][2] - V1[1][2]]
    D = (D1, num_D)
    if cross_prod(V1V2, V1D1) * cross_prod(V1V2, V1V3) > 0
        D = (D2, num_D)
    end
    return [V1, V2, D]
end

make_next (generic function with 1 method)

In [44]:
# A = ([0, 0], 1)
# B = ([1, 0], 2)
# C = ([1 / 2, sqrt(3) / 2], 3)
# edge = (1, 3)
# println(make_next(A, B, C, edge))

In [45]:
function add_point(P::Tuple{AbstractVector{<:Real}, Int64}, points::AbstractVector{Tuple{AbstractVector{Real}, Int64}}, t::Real)
    if (dist(P[1], [0, 0])) > t
        return false
    end
    for check in points
        if (dist(P[1], check[1]) < eps && P[2] == check[2])
            return false
        end
    end
    push!(points, P)
    return true
end

add_point (generic function with 1 method)

In [46]:
function get_all_points(A::AbstractVector{<:Real}, B::AbstractVector{<:Real}, C::AbstractVector{<:Real}, D::AbstractVector{<:Real}, t::Real)
    vertexes = [A, B, C, D]
    for i in 1:4
        for j in 1:4
            ind = (i, j)
            if i == j
                lens[ind] = 0
            else
                lens[ind] = dist3(vertexes[i], vertexes[j])
            end
        end
    end

    Ap = (A[1:2], 1)
    Bp = (B[1:2], 2)
    Cp = (C[1:2], 3)
    start = [Ap, Bp, Cp]
    points = Base.Vector{Tuple{AbstractVector{Real}, Int64}}()
    push!(points, Ap)
    push!(points, Bp)
    push!(points, Cp)
    q = Queue{Tuple{AbstractVector{Tuple{AbstractVector{Real}, Int64}}, Tuple{Int64, Int64}}}()
    enqueue!(q, (start, (0, 0)))
    while length(q) != 0
        info = dequeue!(q)
        face = info[1]
        last_edge = info[2]
        edges = [(face[1][2], face[2][2]), (face[1][2], face[3][2]), (face[2][2], face[3][2])]
        edges = filter(x -> x != last_edge, edges)
        faces = Base.Vector{Tuple{AbstractVector{Tuple{AbstractVector{<:Real}, Int64}}, Tuple{Int64, Int64}}}()
        for edge in edges
            push!(faces, (make_next(face[1], face[2], face[3], edge), edge))
        end
        for info_it in faces
            face_it = info_it[1]
            edge_if = info_it[2]
            flag = false
            for point_tup in face_it
                flag = flag || add_point(point_tup, points, t)
            end
            if flag
                enqueue!(q, info_it)
            end
        end
    end
    return points
end

get_all_points (generic function with 1 method)

In [47]:
function get_geodesics(points::AbstractVector{Tuple{AbstractVector{Real}, Int64}}, precision::Int64)
    result = Dict{Real, Int64}()
    for A in points
        for B in points
            if A[2] == B[2]
                continue
            end
            geodesic_len = round(dist(A[1], B[1]), digits=precision)
            result[geodesic_len] = get(result, geodesic_len, 0) + 1
        end
    end
    return result
end


get_geodesics (generic function with 1 method)

In [48]:
A = [0, 0, 0]
B = [1, 0, 0]
C = [1 / 2, sqrt(3) / 2, 0]
D = [1 / 2, sqrt(3) / 6, sqrt(6) / 3]
get_all_points(A, B, C, D, 14)
# count_lens = Dict{Real, Int64}()

# for double_t in 1:40
#     t = double_t / 2
#     points = get_all_points(A, B, C, D, t)
#     count_lens[t] = length(get_geodesics(points, 3))
#     println("done ", t)
# end
println(1)

0
q
p
p


p
p
p
0.0
q
p
-2.220446049250313e-16
q
p
-4.440892098500626e-16
q
p
-2.220446049250313e-16
q
2.563950248511419e-16
q
p
p
p
7.691850745534257e-16
q
p
p
p
p
p
p
p
0.0
q
p
p
-4.440892098500626e-16
q
1.5543122344752192e-15
q
p
p
-8.881784197001252e-16
q
p
0.0
q
p
0.0
q
1.0255800994045678e-15
q
p
p
p
-1.2819751242557094e-15
q
p
p
p
1.1102230246251565e-15
q
p
p
p
1.9984014443252818e-15
q
p
p
p
6.661338147750939e-16
q
p
p
-8.881784197001252e-16
q
p
-8.881784197001252e-16
q
-3.3306690738754696e-15
q
p
p
0.0
q
8.881784197001252e-16
q
p
p
-8.881784197001252e-16
q
p
-8.881784197001252e-16
q
3.1086244689504383e-15
q
p
p
-8.881784197001252e-16
q
p
p
-7.691850745534256e-16
q
p
p
p
p
p
p
p
-5.127900497022848e-16
q
p
p
p
1.5543122344752192e-15
q
p
p
p
p
p
2.886579864025407e-15


q
p
p
p
p
8.881784197001252e-16
q
8.881784197001252e-16
q
p
p
-8.881784197001252e-16
q
p
-8.881784197001252e-16
q
p
-6.661338147750939e-15
q
3.1086244689504383e-15
q
p
p
8.881784197001252e-16
q
1.3322676295501878e-15
q
p
p
-1.7763568394002505e-15
q
p
-1.7763568394002505e-15
q
-4.440892098500626e-16
q
p
p
-2.6645352591003757e-15
q
p
p
6.1534805964274e-15
q
p
p
p
p
p
p
p
7.179060695831966e-15
q
p
p
p
p
p
1.7763568394002505e-15
q
p
p
p
p
p
p
p
4.440892098500626e-16
q
p
p
p
p
1.7763568394002505e-15
q
5.773159728050814e-15
q
p
p
-3.552713678800501e-15
q
p
-1.7763568394002505e-15
q
p
-1.7763568394002505e-15
q
p
-9.769962616701378e-15
q
6.217248937900877e-15
q
p
p
-1.7763568394002505e-15
q
p
0.0
q
3.1086244689504383e-15
q
p
p
-8.881784197001252e-15
q
p
0.0
q
p
0.0
q


p
2.6645352591003757e-15
q
4.440892098500626e-16
q
p
p
-5.329070518200751e-15
q
-4.884981308350689e-15
q
p
p
p
-3.589530347915983e-15
q
p
p
p
p
p
p
p
1.509903313490213e-14
q
p
p
p
6.153480596427405e-15
q
p
p
p
p
p
1.1102230246251565e-15
q
p
p
p
p
p
p
p
p
p
-2.220446049250313e-15
q
p
p
p
-1.7763568394002505e-15
q
p
p
6.217248937900877e-15
q
-2.220446049250313e-15
q
p
p
-3.552713678800501e-15
q
p
-3.552713678800501e-15
q
p
-1.7763568394002505e-15
q
7.993605777301127e-15
q
p
p
-9.769962616701378e-15
q
2.886579864025407e-15
q
p
p
-7.105427357601002e-15
q
p
-8.881784197001252e-16
q
-3.1086244689504383e-15
q
p
p
-5.329070518200751e-15
q
p
0.0
q
p
0.0
q
p
7.993605777301127e-15
q
-5.329070518200751e-15
q
p
p
-1.0658141036401503e-14
q
-4.440892098500626e-15
q
p
p
p
1.0255800994045642e-15
q
p
p
p
p
p
p
p
3.1086244689504383e-15
q
p
p
p
1.2306961192854846e-14
q
p
p
p
2.6645352591003757e-15
q
p
p
p
-6.8833827526759706e-15
q
p
p
p
p
p
p
p
-2.6645352591003757e-15
q
p
p
p
4.218847493575595e-15
q
p
p
p

-1.0658141036401503e-14
q
8.43769498715119e-15
q
p
p
-8.881784197001252e-15
q
5.329070518200751e-15
q
p
p
-5.329070518200751e-15
q
3.9968028886505635e-15
q
p
p
-3.375077994860476e-14
q
p
-5.329070518200751e-15
q
p
0.0
q
p
0.0
q
-1.7763568394002505e-15
q
p
p
8.881784197001252e-15
q
1.8207657603852567e-14
q
p
p
-1.2434497875801753e-14
q
p
p
-7.549516567451064e-15
q
p
p
p
-3.076740298213699e-15
q
p
p
p
p
p
p
p
p
p
p
p
2.7977620220553945e-14
q
p
p
p
-1.0255800994045653e-14
q
p
p
p
8.881784197001252e-16
q
p
p
p
1.4654943925052066e-14
q
p
p
p
p
p
p
p
p
p
-9.769962616701378e-15
q
p


...

1.0258460747536446e-13
q
p
p
-3.4003022619799594e-11
q
p
-2.8315128020039992e-12
q
p
-2.5934809855243657e-13
q
p
-5.3290705182007514e-14
q
p
-3.552713678800501e-15
q
p
2.842170943040401e-14
q
p
-1.0658141036401503e-14
q
p
1.4566126083082054e-13
q
1.580957587066223e-13
q
p
p
7.105427357601002e-15
q
-1.0480505352461478e-13
q
p
p
-5.3290705182007514e-14
q
-1.0480505352461478e-13
q
p
p
p
5.950795411990839e-14
q
p


p
p
1.2434497875801753e-14
q
p
p
p
-1.128138109345076e-13
q
p
p
p
p
p
p
p
p
p
p
p
-5.1514348342607263e-14
q
p
p
p
-8.881784197001252e-14
q
p
p
p
-1.580957587066223e-13
q
p
p
p
1.0658141036401503e-13
q
p
p
p
4.796163466380676e-14
q
p
p
p
6.394884621840902e-14
q
p
p
p
p
p
p
p
p
p
p
p
-1.1262102361797588e-12
q
p
p
p
1.474376176702208e-13
q
p
p
p
-2.220446049250313e-14
q
p
-8.526512829121202e-14
q
p
p
-3.730349362740526e-14
q
1.0302869668521453e-13
q
p
p
-1.0658141036401503e-14
q
6.306066779870889e-14
q
p
p
-3.318483265957184e-10
q
p
-2.3728574660708546e-11
q
p
-1.957545237019076e-12
q
p
-2.0250467969162855e-13
q
p
-4.618527782440651e-14
q
-7.105427357601002e-14
q
p
p
-4.263256414560601e-14
q
-1.7763568394002505e-13
q
p


p
5.1514348342607263e-14
q
9.237055564881302e-14
q
p
p
-3.197442310920451e-14
q
7.37188088351104e-14
q
p
p
-4.760103422540851e-10
q
p
-3.3999469906120794e-11
q
p
-2.8563817977556027e-12
q
p
-2.7000623958883807e-13
q
p
-6.750155989720952e-14
q
-1.1173284519827575e-12
q
p
p
1.4033219031261979e-13
q
1.0480505352461478e-13
q
p
p
p
-8.526512829121202e-14
q
p
p
p
9.059419880941277e-14
q
p
p
p
p
p
p
p
p
p
-3.197442310920451e-14
q
p
p
p
-1.936228954946273e-13
q
p
p
p
1.0125233984581428e-13
q
p
p
p
8.704148513061227e-14
q
p
p
p
p
p
p
p
-1.0587086762825493e-12
q
p
p
p
p


-7.638334409421077e-14
q
-9.414691248821327e-14
q
p
p
-5.684341886080802e-14
q
p
p
p
-3.318483265957184e-10
q
p
-3.558895400601614e-10
q
-2.1316282072803006e-13
q
p
p
6.394884621840902e-14
q
8.526512829121202e-14
q
p
p
p


DomainError: DomainError with -1.152921504606847e18:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [49]:
println(count_lens)

Dict{Real, Int64}(5.0 => 23, 13.5 => 143, 12.0 => 114, 8.0 => 57, 5.5 => 27, 1.0 => 2, 6.0 => 31, 11.0 => 98, 9.0 => 69, 3.0 => 10, 7.0 => 45, 9.5 => 71, 0.5 => 1, 7.5 => 47, 12.5 => 124, 1.5 => 2, 8.5 => 58, 4.0 => 15, 13.0 => 135, 2.0 => 5, 3.5 => 12, 6.5 => 36, 10.0 => 84, 11.5 => 106, 4.5 => 19, 2.5 => 5, 10.5 => 90)


In [50]:
# using JSON

# io = open("res.json", "w");
# res = Dict{Tuple{Real, Real}, Int64}()
# for point in points
#     res[(point[1][1], point[1][2])] =point[2]
# end
# JSON.print(io, res, 2)
# close(io);

In [51]:
# A1 = [0, 0, 0]
# B1 = [1, 0, 0]
# C1 = [1 / 2, sqrt(3) / 2, 0]
# D1 = [1 / 2, sqrt(3) / 6, sqrt(6) / 2]
# t = 5

# points2 = get_all_points(A1, B1, C1, D1, t)

In [52]:
# using JSON

# io = open("res2.json", "w");
# res = Dict{Tuple{Real, Real}, Int64}()
# for point in points2
#     res[(point[1][1], point[1][2])] = point[2]
# end
# JSON.print(io, res, 2)
# close(io);